# Donkey library tools

Before training our model using the sample data explored in the [introduction chapter](./donkey-intro.ipynb), let's take a look the tools provided by the [Donkey](https://github.com/wroscoe/donkey) library.

This python library has several use-cases. After installation, you can `import` it as a normal python library to create a custom `Donkey` car with components that suite your car:
```python
import donkeycar as dk
```

There is also a handy CLI:
```bash
donkey --help
```

In this chapter, we'll have a look at the CLI, and leave the library for later. The CLI has several subcommands for different tasks. You'll find the complete API documentation here:

- [http://docs.donkeycar.com/utility/donkey/](http://docs.donkeycar.com/utility/donkey/)

## Install

The library is pre-installed on the car, but you need to install it on the [Jupyter Notebook](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-notebooks-instances.html) (or your `host` if you prefer running it locally):

In [ ]:
# Make sure we're in SageMaker root
%cd ~/SageMaker

# Remove any old versions of the library
!rm -rf ~/SageMaker/donkey

In [ ]:
# Clone the Donkey library git
!git clone https://github.com/wroscoe/donkey.git

In [ ]:
# Keras is pinned to version 2.0.8 in the Donkey requirements. Change this to allow a newer version
!sed -i -e 's/keras==2.0.8/keras>=2.1/g' donkey/setup.py

# Install
!pip install -e donkey

In [ ]:
# Test the CLI
!donkey --help

## Download sample data

Download the same sample driving runs as in the [introduction](./donkey-intro.ipynb):

In [ ]:
# Download sample data
sample_data_location = 's3://jayway-robocar-raw-data/samples'
!aws s3 cp {sample_data_location}/ore.zip .

In [ ]:
!unzip -o ore.zip

## Car related commands

Three of the commands relate to the managing and driving of a *Donkey car*:
- `findcar` - Tries to discover the IP-address of your car on the network. The search is restricted to the same subnet as the host (/24), and based on the MAC addresses of Raspberry PIs, hence it will return the IP-address of all Raspberry PIs connected to your subnet (not only Donkey cars).
- `calibrate` - Calibrate *throttle* and *steering* on a newly installed car.
- `createcar` - See [createcar](#createcar)

The `createcar` and `calibrate` commands are used in the [prepare car](../docs/PREPARE-CAR.md) track (see [README](../README.md)), and are further explained there.

The `createcar` command is also used when starting training, so we will have a closer look at it here

### createcar

Create a new *Donkey car application* to the specified *path* based on the specified *template*.

The default *Donkey car application* allows you to **drive** and **train** your car.

A template specifies what *Donkey parts* should be a part of this *Donkey car application*. Templates can be found in `donkey/donkeycar/templates`:

In [ ]:
# This command assumes you've installed the Donkey library to the project root
!ls -l ./donkey/donkeycar/templates

In [ ]:
# Create a new car with the default parameters
!donkey createcar

The default parameters are:
- **`template=donkey2.py`** - Use `donkey2.py` template
- **`path=~/d2`** - Install to the `d2` directory in `$HOME`
- **`config=config_defaults.py`** - Copy the `config_defaults.py` file to `~/d2/config.py` to be used as default config file.

You can have a look at the `donkey2.py` template file just to get a feeling for how the car is set up:

In [ ]:
!cat donkey/donkeycar/templates/donkey2.py

## Tub related commands

Instead, let's have a look at the commands related to *Tubs*:
- `tubcheck`
- `tubclean`
- `tubhist`
- `tubplot`

### tubcheck

Check the *Tub* for invalid records. If `--fix` is `True`, deletes the invalid records.

In [ ]:
!donkey tubcheck tub_8_18-02-09

### tubhist

Produce a histogram of record type frequency in the given tub

In [ ]:
!donkey tubhist tub_8_18-02-09

Note that the histogram cannot be shown in the notebook since the expression that shows the figure, `matplotlib.pyplot.show()`, is run in a subshell. It will simply print the memory location of the figure.

However, we could try to run the python code in the notebook:

In [ ]:
%matplotlib inline
from donkeycar.management.base import ShowHistogram

hst = ShowHistogram()
hst.show_histogram('./tub_8_18-02-09', None)

### tubclean

`tubclean` is a tool for manually examining the *Tub*. It will start a webserver that you can connect to to browse images and metrics.

In [ ]:
!donkey tubclean tub_8_18-02-09

The webserver runs on http://localhost:8886 per default.

**Sadly**, there doesn't seem to be a way to access that port from outside the notebook, hence you'll have to run this on your local host to get it to work.

### tubplot

Plot model predictions for angle and throttle against data from tubs.

**Sadly**, as of 2018-03-21, this command is not implemented correctly.

In [ ]:
!donkey tubplot tub_8_18-02-09

## Video

There is a handly utility for creating a video from the images in the tub.

It requires the `config.py` file used by the car at the time of the recording, which we don't have. But we could try to use the default config file in `donkey/donkeycar/templates/config_defaults.py` instead:

In [ ]:
!donkey makemovie --tub tub_8_18-02-09 --config donkey/donkeycar/templates/config_defaults.py

[Run video](./tub_movie.mp4)

## Simulator

Finally, there is a Donkey car simulator that allows you to both record new data, and run trained models.

![Donkey Simulator](http://www.donkeycar.com/uploads/7/8/1/7/7817903/animation_orig.gif)

- http://docs.donkeycar.com/guide/simulator/

**As with `tubclean`**, the simulator runs as a webserver on your localhost. Thus, there is currently no way of accessing it on a *SageMaker Notebook*.

## Next

[Training the model](./donkey-train.ipynb)